In [26]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import io
import time
from pathlib import Path
import atexit
from scipy.signal import butter, filtfilt
from scipy.stats import linregress
from scipy.signal import find_peaks
from scipy.ndimage import label 
import base64
import os
from pathlib import Path
from tqdm.notebook import tqdm


# ==== CONFIGURATION ====
INPUT_DIR = Path("/Users/Christian/Downloads/Javelin_Bad Krozingen und Neustadt_data")
OUTPUT_FILE = INPUT_DIR / "kinematic_summary.csv"
csv_files = list(INPUT_DIR.glob("*_merged.csv"))
all_results = []


ONLY_LONGEST_ID = True  # Set to False to process all IDs, True for only the one with max rows



fps = 25


# PARAMETERS:
# For javelin leaves the hand: 
WHEN_START_MS = 800 # in milliseconds
MIN_DELTA = 100
MIN_FRACTION = 0.6
MIN_SUSTAIN_SEC = 0.2  # duration in seconds
min_sustain = int(fps * MIN_SUSTAIN_SEC)

THRESHOLD = 6  # how big should the trough be?


# Detecting peaks and troughs for foot-strike:
PROMINENCE = 3
DISTANCE_MS = 300
        

# Stride length: 
STEP_RATIO = 1



# Parameters for if arm is outstretched during the run-up: 
ONSET_OFFSET_MS = 150
OFFSET_BEFORE_RELEASE_MS = 100
WINDOW_BEFORE_RELEASE_MS = 2500
REQUIRED_ABOVE_ANGLE_MS = 150
THRESHOLD_ANGLE = 150
MIN_MAX_DURATION_FRAMES = 2


# Parameters for stopping:
STOP_WIN_MS = 300
STOP_RATIO_THRESHOLD = 0.55

time_window_ms = 400  # Analyze steps at xx ms after release (change as needed)

# Functions: 
def smooth_series(series, window_size=5):
    return pd.Series(series).rolling(window=window_size, min_periods=1, center=True).mean().tolist()



# low butterworth filter
def butter_lowpass_filter(signal, cutoff=5, order=1, fs=30):
    nyquist = 0.5 * fs
    normal_cutoff = cutoff / nyquist
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return filtfilt(b, a, signal)

    

def calculate_angle(ax, ay, bx, by, cx, cy):
    if any(pd.isna([ax, ay, bx, by, cx, cy])):
        return np.nan
    v1, v2 = np.array([ax - bx, ay - by]), np.array([cx - bx, cy - by])
    norm = np.linalg.norm(v1) * np.linalg.norm(v2)
    if norm == 0:
        return np.nan
    angle = np.arccos(np.clip(np.dot(v1, v2) / norm, -1.0, 1.0))
    return np.degrees(angle)

def frame_diff(x, y):
    return [0] + [np.linalg.norm([x[i] - x[i - 1], y[i] - y[i - 1]]) for i in range(1, len(x))]



# Distance function
def dist(wrist_x, wrist_y, obj_x, obj_y):
    if any(pd.isna([wrist_x, wrist_y, obj_x, obj_y])):
        return np.nan
    return np.linalg.norm([wrist_x - obj_x, wrist_y - obj_y])
    



# Split session_name into components
def parse_session_name(session_name):
    parts = session_name.split("_")
    if len(parts) == 5:
        school, class_id, condition, subj_id, throw_nr = parts
    elif len(parts) == 4:
        school = None
        class_id, condition, subj_id, throw_nr = parts
    else:
        return None, None, None, None, None
    try:
        throw_nr = int(str(throw_nr).lstrip("0"))
    except Exception:
        throw_nr = None
    return school, class_id, condition, subj_id, throw_nr



def border_nanmean(arr, n=2, window_size=8):
    """Mean of up to n closest valid values in first/last window_size."""
    arr = np.array(arr)
    # First part
    first = arr[:window_size]
    first_valid = first[np.isfinite(first)]
    if len(first_valid) >= n:
        first_border_mean = np.mean(first_valid[:n])
    elif len(first_valid) > 0:
        first_border_mean = np.mean(first_valid)
    else:
        first_border_mean = np.nan
    # Last part
    last = arr[-window_size:]
    last_valid = last[np.isfinite(last)]
    if len(last_valid) >= n:
        last_border_mean = np.mean(last_valid[-n:])
    elif len(last_valid) > 0:
        last_border_mean = np.mean(last_valid)
    else:
        last_border_mean = np.nan
    return first_border_mean, last_border_mean


def find_sustained_increases(
        signal, deriv, threshold, min_sustain=100, min_delta=MIN_DELTA, min_fraction=MIN_FRACTION, min_start_ms=WHEN_START_MS):
    """
    min_sustain: number of frames
    min_start_ms: where to start searching, in milliseconds (converted to frames)
    fps: frames per second
    """
    min_start = int(round(min_start_ms * fps / 1000))
    i = min_start
    N = len(signal)
    while i <= N - min_sustain - 1:
        if deriv[i] > threshold:
            window = signal[i:i+min_sustain+1]
            start_mean, end_mean = border_nanmean(window, n=2, window_size=5)
            if np.isnan(start_mean) or np.isnan(end_mean):
                i += 1
                continue
            diffs = np.diff(window)
            fraction_increasing = np.mean(diffs > 0)
            sustained = fraction_increasing >= min_fraction
            total_increase = end_mean - start_mean
            if sustained and total_increase > min_delta:
                return np.array([i])
        i += 1
    return np.array([])
    



    

# Loop through files: 
for csv_file in tqdm(csv_files, desc="Processing CSVs"):
    session_name = csv_file.stem.replace("_merged", "")

    school, class_id, condition, subj_id, throw_nr = parse_session_name(session_name)

    if school and school.startswith(("BK", "NE")):
        fps = 25
    else:
        fps = 0

        
    df = pd.read_csv(csv_file)
    person_ids = df["New_ID"].unique()
    
    if ONLY_LONGEST_ID:
        # Find the ID with the most rows
        id_counts = df["New_ID"].value_counts()
        longest_id = id_counts.idxmax()
        person_ids = [longest_id]
    
    
    for person_id in person_ids:
        person_df = df[df["New_ID"] == person_id].copy()
        if person_df.empty:
            continue

        
        
         
        # Prepare plot data
        plot_df = person_df.reset_index(drop=True)
        
        
        # Smoothing the signals: 
        # === Step 1: Setup ===
        joint_names = ["ankle", "knee", "hip", "shoulder", "elbow", "wrist"]
        sides = ["left", "right"]
        axes = ["x", "y"]
        
        # Initialize dictionary to collect all coordinate time series
        coords = {f"{side}_{joint}_{axis}": [] for side in sides for joint in joint_names for axis in axes}
        
        # === Step 2: Fill from plot_df row by row ===
        for row in plot_df.itertuples():
            row_data = row._asdict()
            for key in coords:
                coords[key].append(row_data.get(key, np.nan))
        
        # Define joint motion types
        slow_joints = ["hip", "shoulder"]
        fast_joints = ["ankle", "wrist", "elbow", "Tail", "Handle", "Tip"]
        
        # Apply Butterworth filter with joint-specific cutoffs
        for key in coords:
            signal = coords[key]
            
            if len(signal) > 5 and not all(pd.isna(signal)):
                joint_name = key.split("_")[1] if "_" in key else key  # Extract joint name
        
                # Select cutoff frequency based on joint type
                if any(j in joint_name for j in slow_joints):
                    cutoff = 3
                elif any(j in joint_name for j in fast_joints):
                    cutoff = 7
                else:
                    cutoff = 5
        
                # Try filtering
                try:
                    coords[key] = butter_lowpass_filter(np.array(signal), cutoff=cutoff, order=1, fs=fps)
                except Exception as e:
                    print(f"⚠️ Skipping filter for {key} due to error: {e}")
        
        # === Step 4: Convert to DataFrame ===
        coords_df = pd.DataFrame(coords)
        
        # Optionally add frame column if needed
        coords_df["frame"] = plot_df["Frame"].values
        frame_to_row = {frame: i for i, frame in enumerate(coords_df["frame"])}
        
        
        # Add center coordinates of Tail, Handle, Tip - x and y: 
        # List of javelin parts
        parts = ["Tip", "Handle", "Tail"]
        missing_parts = []
        
        # Add columns, fill with NaN if missing
        for part in parts:
            x_col = f"{part}_center_x"
            y_col = f"{part}_center_y"
            if x_col not in plot_df.columns or y_col not in plot_df.columns:
                missing_parts.append(part)
            coords_df[x_col] = plot_df.get(x_col, pd.Series([np.nan]*len(plot_df))).values
            coords_df[y_col] = plot_df.get(y_col, pd.Series([np.nan]*len(plot_df))).values
        
        if len(missing_parts) == len(parts):
            print(f"⚠️ Warning: ALL javelin parts (Tip, Handle, Tail) missing in {session_name} (will fill with NaN).")
        
        # --- only compute center if at least one point is present ---
        def mean_if_at_least_one(row, cols):
            values = row[cols].values
            not_nan = np.isfinite(values)
            if np.sum(not_nan) >= 1:
                return np.nanmean(values)
            else:
                return np.nan
        
        x_cols = [f"{part}_center_x" for part in parts]
        y_cols = [f"{part}_center_y" for part in parts]
        
        coords_df["javelin_center_x"] = coords_df.apply(lambda row: mean_if_at_least_one(row, x_cols), axis=1)
        coords_df["javelin_center_y"] = coords_df.apply(lambda row: mean_if_at_least_one(row, y_cols), axis=1)
        
        # Distance from javelin center to wrists (same as before)
        coords_df["dist_javelin_to_right_wrist"] = np.sqrt(
            (coords_df["javelin_center_x"] - coords_df["right_wrist_x"])**2 +
            (coords_df["javelin_center_y"] - coords_df["right_wrist_y"])**2
        )
        coords_df["dist_javelin_to_left_wrist"] = np.sqrt(
            (coords_df["javelin_center_x"] - coords_df["left_wrist_x"])**2 +
            (coords_df["javelin_center_y"] - coords_df["left_wrist_y"])**2
        )

        
        # Find instant when javelin leaves the hand:     
        # First derivate to get the sharp increase in javelin to wrist:
        left_signal  = coords_df["dist_javelin_to_left_wrist"].values
        right_signal = coords_df["dist_javelin_to_right_wrist"].values
        
        left_deriv  = np.diff(left_signal, prepend=left_signal[0])
        right_deriv = np.diff(right_signal, prepend=right_signal[0])
        
        
        left_sustained_indices  = find_sustained_increases(
            left_signal, left_deriv, threshold=THRESHOLD, min_sustain=min_sustain, min_delta=MIN_DELTA)
        right_sustained_indices = find_sustained_increases(
            right_signal, right_deriv, threshold=THRESHOLD, min_sustain=min_sustain, min_delta=MIN_DELTA)
        
        
        release_indices = []
        if len(left_sustained_indices) > 0 and len(right_sustained_indices) > 0:
            a = left_sustained_indices[0]
            b = right_sustained_indices[0]
            release_idx = int(np.round((a + b) / 2))
        elif len(left_sustained_indices) > 0:
            release_idx = left_sustained_indices[0]
        elif len(right_sustained_indices) > 0:
            release_idx = right_sustained_indices[0]
        else:
            release_idx = None
        
        if release_idx is None:
            print(f"No release event detected for {session_name} / person {person_id}. Skipping.")
            continue
                
            
        
        # Calculate slope for both hips
        x = np.arange(len(coords_df))  # Frame indices
        
        # Mean hip x for each frame (averaging left and right hip)
        hip_x_mean = (coords_df["left_hip_x"] + coords_df["right_hip_x"]) / 2
        
        # Fit a line: slope tells direction
        slope, intercept, r_value, p_value, std_err = linregress(x, hip_x_mean)
        
        if slope > 0:
            direction = "to_Right"
        elif slope < 0:
            direction = "to_Left"
        else:
            direction = "No Movement"
        
        
        
        # Calculate hip to ankle distance: 
        coords_df["left_ankle_rel_x"] = coords_df["left_hip_x"] - coords_df["left_ankle_x"]
        coords_df["right_ankle_rel_x"] = coords_df["right_hip_x"] - coords_df["right_ankle_x"]
                    
        left_diff = coords_df["left_ankle_rel_x"].values
        right_diff = coords_df["right_ankle_rel_x"].values


        DISTANCE = int(np.round(DISTANCE_MS * fps / 1000))
        
        # Detect extrema type depending on slope
        if slope > 0:
            # Detect minima (lows)
            left_extrema, _ = find_peaks(-left_diff, distance=DISTANCE, prominence=PROMINENCE)
            right_extrema, _ = find_peaks(-right_diff, distance=DISTANCE, prominence=PROMINENCE)
            extrema_label = "Minima (before release)"
        elif slope < 0:
            # Detect maxima (highs)
            left_extrema, _ = find_peaks(left_diff, distance=DISTANCE, prominence=PROMINENCE)
            right_extrema, _ = find_peaks(right_diff, distance=DISTANCE, prominence=PROMINENCE)
            extrema_label = "Maxima (before release)"
        else:
            left_extrema, right_extrema = np.array([]), np.array([])
            extrema_label = "No movement"
        
        # Only those BEFORE the release event
        if release_idx is not None:
            left_extrema_before = left_extrema[left_extrema < release_idx]
            right_extrema_before = right_extrema[right_extrema < release_idx]
        else:
            left_extrema_before, right_extrema_before = [], []
        
        
        
        # Calculations of joint angles:
        
        coords_df["elbow_l"] = coords_df.apply(
            lambda row: calculate_angle(row["left_wrist_x"], row["left_wrist_y"],
                                         row["left_elbow_x"], row["left_elbow_y"],
                                         row["left_shoulder_x"], row["left_shoulder_y"]), axis=1)
        
        coords_df["elbow_r"] = coords_df.apply(
            lambda row: calculate_angle(row["right_wrist_x"], row["right_wrist_y"],
                                         row["right_elbow_x"], row["right_elbow_y"],
                                         row["right_shoulder_x"], row["right_shoulder_y"]), axis=1)
        
        coords_df["knee_l"] = coords_df.apply(
            lambda row: calculate_angle(row["left_ankle_x"], row["left_ankle_y"],
                                         row["left_knee_x"], row["left_knee_y"],
                                         row["left_hip_x"], row["left_hip_y"]), axis=1)
        
        coords_df["knee_r"] = coords_df.apply(
            lambda row: calculate_angle(row["right_ankle_x"], row["right_ankle_y"],
                                         row["right_knee_x"], row["right_knee_y"],
                                         row["right_hip_x"], row["right_hip_y"]), axis=1)
        
        
        
        
        # Calculate stride length: 
        # leg length: 
        leg_lengths_px = []
        for i, row in coords_df.iterrows():
            dists = []
            if row['knee_r'] > 170:
                dx_r = abs(row['right_hip_x'] - row['right_ankle_x'])
                dy_r = abs(row['right_hip_y'] - row['right_ankle_y'])
                d_r = np.sqrt(dx_r**2 + dy_r**2)
                dists.append(d_r)
            if row['knee_l'] > 170:
                dx_l = abs(row['left_hip_x'] - row['left_ankle_x'])
                dy_l = abs(row['left_hip_y'] - row['left_ankle_y'])
                d_l = np.sqrt(dx_l**2 + dy_l**2)
                dists.append(d_l)
            if dists:
                leg_lengths_px.append(np.mean(dists))
        leg_length_px = np.nanmedian(leg_lengths_px)
        
        
        
        
        # 1. Gather events (frame idx, x, y, side)
        contacts = []
        for idx in left_extrema_before:
            contacts.append((idx, coords_df.loc[idx, 'left_ankle_x'], coords_df.loc[idx, 'left_ankle_y'], 'L'))
        for idx in right_extrema_before:
            contacts.append((idx, coords_df.loc[idx, 'right_ankle_x'], coords_df.loc[idx, 'right_ankle_y'], 'R'))
        
        # 2. Sort by frame index (time)
        contacts.sort(key=lambda x: x[0])  # ascending by frame/time
        
        # 3. Only keep those before release, as above (already done if you used *_before arrays)
        # Optional: Go backwards from last before release
        contacts = [c for c in contacts if c[0] < release_idx]
        
        # 4. Calculate all steps before release, regardless of alternation
        step_events = []
        step_lengths_px = []
        step_types = []
        
        for i in range(1, len(contacts)):  # forwards: earliest to latest
            idx_prev, x_prev, y_prev, side_prev = contacts[i-1]
            idx, x, y, side = contacts[i]
            step_length = np.sqrt((x - x_prev)**2 + (y - y_prev)**2)
            step_lengths_px.append(step_length)
            ratio = step_length / leg_length_px if leg_length_px else np.nan
            step_types.append("short" if ratio < STEP_RATIO else "long")
            step_events.append({
                "from_frame": int(idx_prev),
                "to_frame": int(idx),
                "from_side": side_prev,
                "to_side": side,
                "step_length_px": float(step_length),
                "leg_length_px": float(leg_length_px),
                "step_length/leg_length": float(ratio),
                "type": "short" if ratio < 1 else "long"
            })
        
        # Sequence of all foot sides before release
        step_sequence = [c[3] for c in contacts]


        stride_pairs = []
        for i in range(1, len(step_events)):
            prev = step_events[i-1]
            curr = step_events[i]
            if prev["to_side"] != curr["to_side"]:  # alternation (e.g., L to R or R to L)
                stride_pairs.append((prev["to_frame"], curr["to_frame"], prev["to_side"], curr["to_side"]))

    
        # Did the student stop before release?
        # --- Detect steps after release within a restricted time window ---
        # We only count steps that happen shortly after the release event, not later during walking.
        # Adjust `time_window_ms` to set the window (e.g., 400–1200 ms).
        frames_window = int(np.ceil((time_window_ms / 1000) * fps))
        
        # Gather all detected foot contact indices (across both sides, all steps)
        all_contact_indices = np.sort(np.concatenate([left_extrema, right_extrema]))
        
        def steps_after_release_within_window(all_contact_indices, release_idx, frames_window):
            """Return indices of steps occurring after release within a specific window."""
            post_release = all_contact_indices[all_contact_indices > release_idx]
            return [int(idx) for idx in post_release if (idx - release_idx) <= frames_window]
        
        if release_idx is not None:
            post_release_indices_in_window = steps_after_release_within_window(all_contact_indices, release_idx, frames_window)
            if len(post_release_indices_in_window) > 0:
                criterion_release_before_stop = False  # At least one step in window: student did NOT stop immediately after release
                first_post_release_idx = post_release_indices_in_window[0]
            else:
                criterion_release_before_stop = True   # No step in window: student stopped after release
                first_post_release_idx = None
        else:
            criterion_release_before_stop = None
            first_post_release_idx = None
        
        
        if first_post_release_idx is not None:
            first_post_release_ms = (first_post_release_idx - release_idx) * 1000 / fps
        else:
            first_post_release_ms = None
        
        
        

        # --- Deceleration Detection: Is there a "strong stop" before release? ---    
        # Safe defaults in case of early exit
        vel_before = np.nan
        vel_stop = np.nan
        ratio = np.nan
        strong_stop = None
        stop_accel_value = None
        stop_vel_after = None
        
        # Only calculate if release_idx is valid
        if release_idx is not None:
            coords_df["mean_hip_x"] = (coords_df["left_hip_x"] + coords_df["right_hip_x"]) / 2
        
            # Velocity per frame (pixels/frame)
            coords_df["mean_hip_x_vel"] = coords_df["mean_hip_x"].diff().fillna(0)
            # Convert to velocity in pixels/sec
            coords_df["mean_hip_x_vel"] *= fps
        
            # Frame indices for windows
            onset_offset_frames = int(np.ceil(ONSET_OFFSET_MS / 1000 * fps))
            stop_win_frames = int(STOP_WIN_MS / 1000 * fps)
        
            # Indices for stop window (500 ms before release up to release)
            idx_stop_win_start = max(onset_offset_frames, release_idx - stop_win_frames)
            idx_stop_win_end = release_idx + 1  # include release frame
        
            # Indices for reference window (max 2.5s before stop window, starting at 150 ms after onset)
            idx_runup_end = idx_stop_win_start
            idx_runup_start = max(onset_offset_frames, idx_runup_end - int(WINDOW_BEFORE_RELEASE_MS / 1000 * fps))
        
            # Use mean hip x velocity series
            hip_vel_series = coords_df["mean_hip_x_vel"].values
        
            # Calculate mean velocities in the windows
            vel_before = np.abs(hip_vel_series[idx_runup_start:idx_runup_end]).mean() if idx_runup_end > idx_runup_start else np.nan
            vel_stop   = np.abs(hip_vel_series[idx_stop_win_start:idx_stop_win_end]).mean() if idx_stop_win_end > idx_stop_win_start else np.nan
        
            # Stop ratio and criterion
            ratio = vel_stop / vel_before if vel_before > 0 else np.nan
            strong_stop = ratio < STOP_RATIO_THRESHOLD if not np.isnan(ratio) else None
        
            # --- Additional: Compute hip acceleration (stop_accel_value) and post-release velocity (stop_vel_after) ---
            # Hip acceleration (pixels/sec²)
            hip_acc_series = np.diff(hip_vel_series, prepend=hip_vel_series[0]) * fps
        
            # Minimum acceleration (strongest deceleration) in stop window
            if idx_stop_win_end > idx_stop_win_start:
                stop_accel_value = np.nanmin(hip_acc_series[idx_stop_win_start:idx_stop_win_end])
            else:
                stop_accel_value = None
        
            # Mean hip velocity in window after release (e.g., next 200 ms)
            POST_WIN_MS = 200
            post_win_frames = int(np.ceil(POST_WIN_MS / 1000 * fps))
            idx_post_win_start = release_idx + 1
            idx_post_win_end = min(len(hip_vel_series), release_idx + 1 + post_win_frames)
        
            if idx_post_win_end > idx_post_win_start:
                stop_vel_after = np.abs(hip_vel_series[idx_post_win_start:idx_post_win_end]).mean()
            else:
                stop_vel_after = None


        
        
        
        
        
        # Calculation for assessing if elbow was pointing in the throwing direction
        # PRE window: exactly 500 ms
        
        n_pre_frames = int(np.round(0.500 * fps))
        idx_pre_start = max(0, release_idx - n_pre_frames + 1)
        idx_pre_end = release_idx + 1
        pre_window = coords_df.iloc[idx_pre_start:idx_pre_end]
        
        # POST window: exactly 500 ms
        n_post_frames = int(np.round(0.500 * fps))
        idx_post_start = release_idx
        idx_post_end = min(len(coords_df), release_idx + n_post_frames)
        post_window = coords_df.iloc[idx_post_start:idx_post_end]
        
        if direction == "to_Right":
            throwing_side = "right"
        else:
            throwing_side = "left"
        
        elbow_x_col = f"{throwing_side}_elbow_x"
        wrist_x_col = f"{throwing_side}_wrist_x"
        elbow_angle_col = f"elbow_{throwing_side[0]}"  # 'elbow_r' or 'elbow_l'
        
        
        # 1. Elbow ahead of wrist (20%+)
        if direction == "to_Right":
            ahead_mask = pre_window[elbow_x_col] > pre_window[wrist_x_col]
        else:
            ahead_mask = pre_window[elbow_x_col] < pre_window[wrist_x_col]
        
        frac_ahead = np.mean(ahead_mask)
        criterion_1 = frac_ahead >= 0.20
        
        # 2. Min elbow angle <90° in at least 3% of frames
        frac_min_angle = np.mean(pre_window[elbow_angle_col] < 90)
        criterion_2 = frac_min_angle >= 0.03
        # For min elbow angle < 90° (pre-release)
        num_frames_min_angle = np.sum(pre_window[elbow_angle_col] < 90)
        duration_min_angle_ms = num_frames_min_angle * (1000 / fps)
        
        # 3. Max elbow angle >150° in at least 3% of frames after release
        frac_max_angle = np.mean(post_window[elbow_angle_col] > 150)
        criterion_3 = frac_max_angle >= 0.03
        # For max elbow angle > 150° (post-release)
        num_frames_max_angle = np.sum(post_window[elbow_angle_col] > 150)
        duration_max_angle_ms = num_frames_max_angle * (1000 / fps)
        
        # 4. Positive slope of elbow angle before release 
        slope_win = int(0.300 * fps)
        idx_slope_start = max(0, release_idx - slope_win)
        idx_slope_end = release_idx + 1
        
        elbow_angles_slope = coords_df[elbow_angle_col].iloc[idx_slope_start:idx_slope_end].values
        x_vals = np.arange(len(elbow_angles_slope))
        
        if len(elbow_angles_slope) > 1:
            slope_val, _, _, _, _ = linregress(x_vals, elbow_angles_slope)
            criterion_4 = slope_val > 0
        else:
            slope_val = np.nan
            criterion_4 = False
        
        # For min in pre-window
        tol = 5  # or whatever you use for float tolerance
        min_val = np.nanmin(pre_window[elbow_angle_col])
        min_mask = np.abs(pre_window[elbow_angle_col] - min_val) < tol
        labeled, n_features = label(min_mask)
        durations = [np.sum(labeled == i) for i in range(1, n_features + 1)]
        if any(d >= 3 for d in durations):
            min_angle_valid = min_val
        else:
            min_angle_valid = np.nan
        
        # For max in post-window
        max_val = np.nanmax(post_window[elbow_angle_col])
        max_mask = np.abs(post_window[elbow_angle_col] - max_val) < tol
        labeled, n_features = label(max_mask)
        durations = [np.sum(labeled == i) for i in range(1, n_features + 1)]
        if any(d >= 3 for d in durations):
            max_angle_valid = max_val
        else:
            max_angle_valid = np.nan
        
        
        num_frames_ahead = np.sum(ahead_mask)
        
    
        # Elbow outstretched AND behind shoulder
        
        # Determine correct elbow and shoulder columns based on throwing side
        if direction == "to_Right":
            throwing_side = "right"
            elbow_x_col = "right_elbow_x"
            shoulder_x_col = "right_shoulder_x"
            elbow_angle_col = "elbow_r"
        elif direction == "to_Left":
            throwing_side = "left"
            elbow_x_col = "left_elbow_x"
            shoulder_x_col = "left_shoulder_x"
            elbow_angle_col = "elbow_l"
        else:
            raise ValueError("Unknown throwing direction!")
        
        
  
        
        # Calculate frame indices
        onset_offset_frames = int(np.ceil(ONSET_OFFSET_MS / 1000 * fps))
        offset_frames = int(np.round(OFFSET_BEFORE_RELEASE_MS / 1000 * fps))
        idx_win_end = max(onset_offset_frames, release_idx - offset_frames + 1)
        idx_win_start = max(onset_offset_frames, idx_win_end - int(WINDOW_BEFORE_RELEASE_MS / 1000 * fps))
        
        elbow_series = coords_df[elbow_angle_col].iloc[idx_win_start:idx_win_end].values
        elbow_x_series = coords_df[elbow_x_col].iloc[idx_win_start:idx_win_end].values
        shoulder_x_series = coords_df[shoulder_x_col].iloc[idx_win_start:idx_win_end].values
        
        # 1. Frames with sustained extension AND elbow behind shoulder
        above = elbow_series > THRESHOLD_ANGLE
        if direction == "to_Right":
            behind = elbow_x_series < shoulder_x_series
        elif direction == "to_Left":
            behind = elbow_x_series > shoulder_x_series
        else:
            behind = np.ones_like(elbow_x_series, dtype=bool)  # fallback: don't restrict
        
        above_and_behind = above & behind
        num_above_and_behind = np.sum(above_and_behind)
        duration_above_and_behind_ms = num_above_and_behind * 1000 / fps
        sustained_extension_and_behind = duration_above_and_behind_ms >= REQUIRED_ABOVE_ANGLE_MS

        if sustained_extension_and_behind:
            above_indices = np.where(above_and_behind)[0]
            sustained_start_idx = idx_win_start + above_indices[0]
            sustained_end_idx = idx_win_start + above_indices[-1]
        else:
            sustained_start_idx = sustained_end_idx = np.nan
        
        # 2. Maximum angle reached (descriptive, not a pass/fail)
        max_angle = np.nanmax(elbow_series)
        
        # 3. (Optional/Descriptive) Did max angle persist for at least N frames?
        from scipy.ndimage import label
        tolerance = 5
        max_mask = np.abs(elbow_series - max_angle) < tolerance
        labeled, n_features = label(max_mask)
        max_durations = [np.sum(labeled == i) for i in range(1, n_features + 1)]
        max_angle_streak = max(max_durations) if max_durations else 0
        max_angle_streak_ok = max_angle_streak >= MIN_MAX_DURATION_FRAMES


        if release_idx is None:
            print(f"No release event detected for {session_name} / person {person_id}. Skipping.")
            continue
        summary_row = {
            "session": session_name,
            "person_id": person_id,
            "school": school, 
            "class_id": class_id,
            "condition": condition,
            "subject_id": subj_id,
            "throw_number": throw_nr,
            "release_idx": int(release_idx),
            "direction": direction,
            "fps": fps, 
            "slope": float(slope),
            "leg_length_px": float(leg_length_px),
            "num_steps": len(step_events),
            "step_from_frames": [x["from_frame"] for x in step_events],
            "step_to_frames": [x["to_frame"] for x in step_events],
            "step_from_sides": [x["from_side"] for x in step_events],
            "step_to_sides": [x["to_side"] for x in step_events],
            "step_lengths_px": [x["step_length_px"] for x in step_events],
            "step_types": [x["type"] for x in step_events],
            "step_sequence": [x["to_side"] for x in step_events], 
            # Real event-based kinematic fields (replace with your calculated variables)
            "throwing_side": throwing_side,
            "elbow_ahead_of_wrist_ms_pre": float(num_frames_ahead * 1000 / fps),
            "criterion_1_elbow_ahead": bool(criterion_1),
            "elbow_below_90_ms_pre": float(duration_min_angle_ms),
            "criterion_2_min_elbow": bool(criterion_2),
            "elbow_above_150_ms_post": float(duration_max_angle_ms),
            "criterion_3_max_elbow": bool(criterion_3),
            "elbow_angle_slope": float(slope_val) if not np.isnan(slope_val) else np.nan,
            "criterion_4_positive_slope": bool(criterion_4),
            "min_elbow_angle_pre": float(min_angle_valid),
            "max_elbow_angle_post": float(max_angle_valid),
            "max_elbow_angle_pre_release": float(max_angle),
            "max_angle_streak_frames": int(max_angle_streak),
            "sustained_extension": bool(sustained_extension_and_behind),
            "sustained_extension_start_idx": sustained_start_idx,
            "sustained_extension_end_idx": sustained_end_idx,
            "true_stop_after_release": bool(criterion_release_before_stop),
            "first_post_release_idx": int(first_post_release_idx) if first_post_release_idx is not None else None,
            "first_post_release_ms": float(first_post_release_ms) if first_post_release_ms is not None else None,
            "strong_stop": bool(strong_stop) if strong_stop is not None else None,
            "vel_before": float(vel_before) if not np.isnan(vel_before) else None,
            "vel_stop": float(vel_stop) if not np.isnan(vel_stop) else None,
            "stop_ratio": float(ratio) if not np.isnan(ratio) else None,
            "min_hip_acc": float(stop_accel_value) if stop_accel_value is not None else None,
            "stop_vel_after": float(stop_vel_after) if stop_vel_after is not None else None,
            
        }
        all_results.append(summary_row)
    

summary_df = pd.DataFrame(all_results)
summary_df.to_csv(OUTPUT_FILE, index=False)
print(f"Saved summary to {OUTPUT_FILE}")


Processing CSVs:   0%|          | 0/1884 [00:00<?, ?it/s]

No release event detected for NE_8c_ps1_104_02 / person 1. Skipping.
No release event detected for BK_8a_I_23_03 / person 3. Skipping.
No release event detected for NE_8c_post_87_03 / person 1. Skipping.
No release event detected for NE_8b_pre_54_04 / person 1. Skipping.
No release event detected for NE_8c_I_86_01 / person 1. Skipping.
No release event detected for BK_8c_ps3_72_03 / person 1. Skipping.
No release event detected for BK_8c_ps2_71_04 / person 1. Skipping.
No release event detected for BK_8a_pre_05_02 / person 1. Skipping.
No release event detected for BK_8b_post_31_01 / person 1. Skipping.
No release event detected for BK_8b_post_38_03 / person 1. Skipping.
No release event detected for BK_8a_I_10_04 / person 1. Skipping.
No release event detected for BK_8b_ps1_31_02 / person 1. Skipping.
No release event detected for BK_8c_ps3_65_03 / person 1. Skipping.
No release event detected for BK_8b_ps2_35_04 / person 1. Skipping.
No release event detected for NE_8a_I_05_02 / pers

/opt/miniconda3/envs/myenv/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


No release event detected for BK_8c_ps3_61_03 / person 1. Skipping.
No release event detected for BK_8b_ps2_31_04 / person 1. Skipping.
No release event detected for BK_8b_ps1_35_02 / person 1. Skipping.
No release event detected for NE_8c_pre_84_01 / person 1. Skipping.
No release event detected for BK_8a_I_03_04 / person 1. Skipping.
No release event detected for BK_8a_pre_01_02 / person 2. Skipping.
No release event detected for NE_8a_I_09_03 / person 1. Skipping.
No release event detected for BK_8b_I_37_03 / person 1. Skipping.
No release event detected for BK_8b_ps1_34_01 / person 1. Skipping.
No release event detected for BK_8c_ps1_67_01 / person 1. Skipping.
No release event detected for NE_8c_ps1_100_02 / person 1. Skipping.
No release event detected for NE_8a_I_16_02 / person 1. Skipping.
No release event detected for NE_8c_I_82_01 / person 1. Skipping.
No release event detected for NE_8a_I_03_04 / person 1. Skipping.
No release event detected for NE_8c_pre_90_04 / person 1. S

/opt/miniconda3/envs/myenv/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


...

No release event detected for BK_8a_ps1_22_02 / person 2. Skipping.
No release event detected for NE_8c_ps4_84_04 / person 1. Skipping.
No release event detected for NE_8b_ps4_51_02 / person 1. Skipping.
No release event detected for BK_8a_ps2_25_01 / person 1. Skipping.
No release event detected for BK_8b_post_53_01 / person 1. Skipping.
No release event detected for BK_8a_ps4_24_03 / person 1. Skipping.
No release event detected for NE_8c_ps3_86_01 / person 1. Skipping.
No release event detected for BK_8c_I_61_02 / person 2. Skipping.
No release event detected for NE_8a_ps2_07_01 / person 1. Skipping.
No release event detected for NE_8b_ps4_47_01 / person 1. Skipping.
No release event detected for NE_8a_ps4_06_03 / person 1. Skipping.
No release event detected for BK_8b_ps3_54_03 / person 2. Skipping.
No release event detected for BK_8a_ps2_24_02 / person 1. Skipping.
No release event detected for BK_8a_ps1_04_03 / person 1. Skipping.
No release event detected for NE_8a_ps4_18_01 / p

/opt/miniconda3/envs/myenv/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


No release event detected for BK_8b_ps2_50_01 / person 1. Skipping.
No release event detected for BK_8b_ps1_42_04 / person 1. Skipping.
No release event detected for BK_8b_post_40_01 / person 1. Skipping.
No release event detected for BK_8a_ps1_24_04 / person 1. Skipping.
No release event detected for NE_8c_ps1_84_03 / person 1. Skipping.
No release event detected for BK_8a_post_22_03 / person 1. Skipping.
No release event detected for BK_8c_I_72_02 / person 1. Skipping.
No release event detected for BK_8a_post_06_04 / person 1. Skipping.
No release event detected for NE_8a_ps1_06_04 / person 1. Skipping.
No release event detected for NE_8c_ps2_103_03 / person 1. Skipping.
No release event detected for BK_8b_ps1_48_03 / person 1. Skipping.
No release event detected for BK_8b_ps2_53_04 / person 2. Skipping.
No release event detected for NE_8a_ps4_15_03 / person 1. Skipping.
No release event detected for BK_8b_ps3_50_03 / person 1. Skipping.
No release event detected for BK_8b_ps1_41_01 

/opt/miniconda3/envs/myenv/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


No release event detected for NE_8a_ps2_15_02 / person 1. Skipping.
No release event detected for NE_8c_ps3_82_01 / person 1. Skipping.
No release event detected for BK_8a_ps3_10_02 / person 1. Skipping.
No release event detected for BK_8b_ps1_40_02 / person 1. Skipping.
Saved summary to /Users/Christian/Downloads/Javelin_Bad Krozingen und Neustadt_data/kinematic_summary.csv
